# Consultar datos de MySQL e cargalos en MongoDB (Notebook 4)

## Desde MySQL, selecciona a táboa de datos inserida.

## Conéctate á base de datos MongoDB usando Python.

## Crea unha colección.

## Insire documentos a partir de filas do teu DataFrame.

Verifica realizando consultas simples (find, count, etc.).